# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kerikeri,-35.2268,173.9474,24.31,63,17,3.28,NZ,1707364509
1,1,waitangi,-43.9535,-176.5597,19.34,84,100,2.04,NZ,1707364509
2,2,port-aux-francais,-49.3500,70.2167,4.31,70,82,11.96,TF,1707364509
3,3,mpanda,-6.3655,31.0409,17.27,96,34,0.52,TZ,1707364509
4,4,dakoro,14.5106,6.7650,14.10,22,0,4.62,NE,1707364509


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600,
    color = "Country"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrowed = city_data_df.loc[(city_data_df["Max Temp"] <= 30) &
                                      (city_data_df["Max Temp"] >= 20) &
                                      (city_data_df["Wind Speed"] <= 4.0) &
                                      (city_data_df["Cloudiness"] <= 0.1)]
# Drop any rows with null values
city_data_dropped = city_data_narrowed.dropna(how="any")

# Display sample data
city_data_dropped

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,margaret river,-33.9500,115.0667,26.20,44,0,2.33,AU,1707364500
234,234,lionel town,17.8101,-77.2406,23.94,74,0,2.23,JM,1707364541
265,265,thongwa,16.7619,96.5278,27.09,39,0,1.57,MM,1707364546
276,276,punta del este,-34.9667,-54.9500,25.06,82,0,3.13,UY,1707364547
325,325,touba,8.2833,-7.6833,22.57,22,0,1.09,CI,1707364553
333,333,phongsali,21.6833,102.1058,22.82,30,0,1.16,LA,1707364555
349,349,dakhla,23.6848,-15.9580,20.12,38,0,3.66,EH,1707364557
416,416,les cayes,18.2000,-73.7500,20.02,66,0,3.36,HT,1707364567
466,466,thames,-37.1333,175.5333,27.87,41,0,2.76,NZ,1707364573


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_dropped[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,margaret river,AU,-33.9500,115.0667,44,
234,lionel town,JM,17.8101,-77.2406,74,
265,thongwa,MM,16.7619,96.5278,39,
276,punta del este,UY,-34.9667,-54.9500,82,
325,touba,CI,8.2833,-7.6833,22,
333,phongsali,LA,21.6833,102.1058,30,
349,dakhla,EH,23.6848,-15.9580,38,
416,les cayes,HT,18.2000,-73.7500,66,
466,thames,NZ,-37.1333,175.5333,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

category = "accommodation.hotel"

params = {
    "categories": category,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
lionel town - nearest hotel: No hotel found
thongwa - nearest hotel: No hotel found
punta del este - nearest hotel: Hotel Aquarium
touba - nearest hotel: Hôtel de l'amitié vrai
phongsali - nearest hotel: Phonsaly Hotel
dakhla - nearest hotel: Hotel El Massira
les cayes - nearest hotel: No hotel found
thames - nearest hotel: Kopu Station Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
27,margaret river,AU,-33.9500,115.0667,44,Margaret River Hotel
234,lionel town,JM,17.8101,-77.2406,74,No hotel found
265,thongwa,MM,16.7619,96.5278,39,No hotel found
276,punta del este,UY,-34.9667,-54.9500,82,Hotel Aquarium
325,touba,CI,8.2833,-7.6833,22,Hôtel de l'amitié vrai
333,phongsali,LA,21.6833,102.1058,30,Phonsaly Hotel
349,dakhla,EH,23.6848,-15.9580,38,Hotel El Massira
416,les cayes,HT,18.2000,-73.7500,66,No hotel found
466,thames,NZ,-37.1333,175.5333,41,Kopu Station Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600,
    color = "Country",
    hover_cols='all'
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,index,City,Hotel Name)